## Modelo de recomendación

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [4]:
games = pd.read_csv('games.csv', sep=',')

In [5]:
games_model = games.copy()
games_model = games_model.drop(['release_date', 'app_name', 'publisher', 'developer', 'early_access'], axis=1)

In [6]:
games_model['genres'] = games_model['genres'].apply(ast.literal_eval)
games_model['tags'] = games_model['tags'].apply(ast.literal_eval)
games_model['specs'] = games_model['specs'].apply(ast.literal_eval)

In [7]:
games_model['genres'] = games_model['genres'].apply(lambda x: ', '.join(x))
games_model['tags'] = games_model['tags'].apply(lambda x: ', '.join(x))
games_model['specs'] = games_model['specs'].apply(lambda x: ', '.join(x))

In [8]:
games_model.head(1)

,genres,title,url,tags,reviews_url,specs,price,id
0,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"Strategy, Action, Indie, Casual, Simulation",http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,761140.0


In [9]:
def clean_text(tags):
    result = str(tags).lower()
    return(result.replace(',',' '))

In [10]:
games_model['tags'] = games_model['tags'].apply(clean_text)
games_model['genres'] = games_model['genres'].apply(clean_text)
games_model['specs'] = games_model['specs'].apply(clean_text)

In [11]:
def clean_text2(specs):
    result = str(specs).lower()
    return(result.replace('-',''))

In [12]:
games_model['tags'] = games_model['tags'].apply(clean_text2)
games_model['genres'] = games_model['genres'].apply(clean_text2)
games_model['specs'] = games_model['specs'].apply(clean_text2)

In [13]:
games_model = games_model[['url', 'id', 'genres', 'tags', 'specs', 'price']]
games_model['id'] = games_model['id'].astype(int)
games_model.head(1)

,url,id,genres,tags,specs,price
0,http://store.steampowered.com/app/761140/Lost_...,761140,action casual indie simulation strategy,strategy action indie casual simulation,singleplayer,4.99


In [29]:
games_model.to_csv('./API_csv/ML_games_model.csv', index=False)

In [14]:
games_selected2 = games_model.drop(['url'],axis=1)

games_selected2 = games_selected2[games_selected2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
print(games_selected2.shape)
print(games_selected2.head())

(32131,)
0    action  casual  indie  simulation  strategy st...
1    free to play  indie  rpg  strategy free to pla...
2    casual  free to play  indie  simulation  sport...
3    action  adventure  casual action  adventure  c...
4    no info available action  indie  casual  sport...
dtype: object


In [34]:
type(games_selected2)

pandas.core.series.Series

In [28]:
games_selected2.to_csv('./API_csv/ML_games_selected2.csv', index=False, header=False)

In [15]:
vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(games_selected2)

In [16]:
vectorized

<32131x540 sparse matrix of type '<class 'numpy.int64'>'
	with 477725 stored elements in Compressed Sparse Row format>

In [17]:
similarities = cosine_similarity(vectorized)
print(similarities)

[[1.         0.19583736 0.36710862 ... 0.46524211 0.5        0.34815531]
 [0.19583736 1.         0.59481188 ... 0.28953301 0.23935678 0.10416667]
 [0.36710862 0.59481188 1.         ... 0.26013299 0.28239125 0.16222142]
 ...
 [0.46524211 0.28953301 0.26013299 ... 1.         0.55829053 0.35634832]
 [0.5        0.23935678 0.28239125 ... 0.55829053 1.         0.3046359 ]
 [0.34815531 0.10416667 0.16222142 ... 0.35634832 0.3046359  1.        ]]


In [16]:
games_vect = pd.DataFrame(similarities, columns=games_model['id'], index=games_model['id']).reset_index()
games_vect

id,id,761140,643980,670290,767400,773570,772540,774276,774277,774278,...,771810,767590,747320,769330,745400,773640,733530,610660,658870,681550
0,761140,1.000000,0.195837,0.367109,0.569803,0.381385,0.426401,0.264443,0.262432,0.262432,...,0.465379,0.454545,0.286039,0.578542,0.596962,0.858116,0.574427,0.465242,0.500000,0.348155
1,643980,0.195837,1.000000,0.594812,0.027277,0.068465,0.081650,0.607644,0.603023,0.603023,...,0.123768,0.217597,0.319505,0.153864,0.204124,0.273861,0.314270,0.289533,0.239357,0.104167
2,670290,0.367109,0.594812,1.000000,0.176998,0.207322,0.132453,0.706438,0.701066,0.701066,...,0.160623,0.254152,0.148087,0.359425,0.238416,0.385027,0.254906,0.260133,0.282391,0.162221
3,767400,0.569803,0.027277,0.176998,1.000000,0.358569,0.534522,0.066299,0.065795,0.065795,...,0.583383,0.341882,0.119523,0.402911,0.748331,0.358569,0.308607,0.116642,0.398862,0.327327
4,773570,0.381385,0.068465,0.207322,0.358569,1.000000,0.178885,0.249615,0.247717,0.247717,...,0.325396,0.429058,0.200000,0.337100,0.357771,0.300000,0.258199,0.195180,0.333712,0.684653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32126,773640,0.858116,0.273861,0.385027,0.358569,0.300000,0.357771,0.416025,0.412861,0.412861,...,0.271163,0.619751,0.450000,0.606780,0.581378,1.000000,0.774597,0.634335,0.667424,0.319505
32127,733530,0.574427,0.314270,0.254906,0.308607,0.258199,0.230940,0.501280,0.497468,0.497468,...,0.233380,0.738549,0.602464,0.522233,0.692820,0.774597,1.000000,0.545949,0.779579,0.353553
32128,610660,0.465242,0.289533,0.260133,0.116642,0.195180,0.392792,0.568399,0.564076,0.564076,...,0.264628,0.511766,0.536745,0.328976,0.480079,0.634335,0.545949,1.000000,0.558291,0.356348
32129,658870,0.500000,0.239357,0.282391,0.398862,0.333712,0.255841,0.449553,0.446134,0.446134,...,0.310253,0.727273,0.619751,0.642824,0.724882,0.667424,0.779579,0.558291,1.000000,0.304636


In [31]:
games_u_p = games.drop(['publisher','genres','app_name','id','release_date','tags','reviews_url','specs', 'early_access', 'developer'], axis=1)
games_u_p = games_u_p.reset_index()
games_u_p.head(1)

,index,title,url,price
0,0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,4.99


In [32]:
games_u_p.to_csv('./API_csv/ML_games_u_p.csv', index=False)

In [18]:
games_concat = pd.concat([games_u_p, games_vect], axis=1)

In [19]:
games_concat.head()

,index,title,url,price,id,761140,643980,670290,767400,773570,...,771810,767590,747320,769330,745400,773640,733530,610660,658870,681550
0,0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,4.99,761140,1.000000,0.195837,0.367109,0.569803,0.381385,...,0.465379,0.454545,0.286039,0.578542,0.596962,0.858116,0.574427,0.465242,0.500000,0.348155
1,1,Ironbound,http://store.steampowered.com/app/643980/Ironb...,Free To Play,643980,0.195837,1.000000,0.594812,0.027277,0.068465,...,0.123768,0.217597,0.319505,0.153864,0.204124,0.273861,0.314270,0.289533,0.239357,0.104167
2,2,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,Free to Play,670290,0.367109,0.594812,1.000000,0.176998,0.207322,...,0.160623,0.254152,0.148087,0.359425,0.238416,0.385027,0.254906,0.260133,0.282391,0.162221
3,3,弹炸人2222,http://store.steampowered.com/app/767400/2222/,0.99,767400,0.569803,0.027277,0.176998,1.000000,0.358569,...,0.583383,0.341882,0.119523,0.402911,0.748331,0.358569,0.308607,0.116642,0.398862,0.327327
4,4,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,2.99,773570,0.381385,0.068465,0.207322,0.358569,1.000000,...,0.325396,0.429058,0.200000,0.337100,0.357771,0.300000,0.258199,0.195180,0.333712,0.684653


In [20]:
games_concat.to_csv('./API_csv/model_recommendation.csv')

In [21]:
def game_recommendation(id: int):
    try:
        print('Wait a minute and you will see our recommendations. Enjoy! ')
        recommendations = pd.DataFrame(games_concat.nlargest(6,id)['id'])
        recommendations = recommendations[recommendations['id']!=id]
        title = pd.DataFrame(games_concat.nlargest(6,id)['title'])
        price = pd.DataFrame(games_concat.nlargest(6,id)['price'])
        url = pd.DataFrame(games_concat.nlargest(6,id)['url'])
        result2 = pd.concat([recommendations,title, price[1:11],url[1:]], axis = 1)
        print(result2)
    except:
        print('Sorry, we can not find a suitable match. Try a different game! ')

In [22]:
game_recommendation(670290)

Wait a minute and you will see our recommendations. Enjoy! 
             id                                     title         price  \
7260   522660.0  Snooker-online multiplayer snooker game!  Free To Play   
10443  645260.0                          Trophy Fishing 2  Free To Play   
8221   542940.0                           Kickoff Legends  Free To Play   
8103   402560.0                                Tabletopia  Free to Play   
7695   458960.0                 Winning Putt: Golf Online  Free to Play   
2           NaN                   Real Pool 3D - Poolians           NaN   

                                                     url  
7260   http://store.steampowered.com/app/522660/Snook...  
10443  http://store.steampowered.com/app/645260/Troph...  
8221   http://store.steampowered.com/app/542940/Kicko...  
8103   http://store.steampowered.com/app/402560/Table...  
7695   http://store.steampowered.com/app/458960/Winni...  
2                                                    NaN  
